In [1]:
%cd "C:\Users\user\Documents\UST\BDT\MSBD5005\project\data\"

C:\Users\user\Documents\UST\BDT\MSBD5005\project\data


In [2]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
!pip install snscrape
!pip install wheel
!pip install --upgrade nbformat
!pip install plotly
# !pip install flair

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\user\appdata\local\temp\pip-req-build-7x4wh56d


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import re
import pickle
import snscrape
import math
import plotly.express as px
import plotly.graph_objects as go
import pickle
import datetime
from datetime import date, timedelta

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

read data

In [4]:
file = './daily/tweet_verified_221130-230418_with_sentiment.pkl'

with open(file, 'rb') as f:
    tweets_dict = pickle.load(f)

In [5]:
# remove occurance of r in s, mainly for removing '\n'
def removeAll(s: str, r: str) -> str:
    while r in s:
        s = s.replace(r, ' ')
    return s

In [6]:
arr = []
df_column = ['tid', 'rawContent', 'renderedContent', 'datetime', 'date', 'scrapDate', 'retweetCount', 'viewCount', 'likeCount', 'sentiment']

for tid, tweet in tweets_dict.items():
    rawContent = removeAll(tweet.rawContent, '\n')
    renderedContent = removeAll(tweet.renderedContent, '\n')
    _datetime = tweet.date
    _date = _datetime.date()
    scrapDate = datetime.date(2023, 4, 18)
    retweetCount = tweet.retweetCount
    viewCount = tweet.viewCount if tweet.viewCount else 0
    likeCount = tweet.likeCount if tweet.likeCount else 0
    sentiment = tweet.sentiment
    arr.append((tid, rawContent, renderedContent, _datetime, _date, scrapDate, retweetCount, viewCount, likeCount, sentiment))

df = pd.DataFrame(arr, columns=df_column)

# remove tweets that are before the first scrap date
target_date = datetime.date.fromisoformat('2023-03-28')
df = df[df['date'] > target_date]
print(len(df))

11910


In [7]:
# select tweets with top n tweet with certain attribute everyday

topk = 5
topAttribute = 'viewCount'
df1 = df.groupby(by=['date']).apply(lambda x: x.nlargest(topk ,[topAttribute])).reset_index(drop=True)
print(len(df1))

105


In [8]:
# use tid as key to create attribute dictionary

tid_list = list(df1['tid'])
rawContent_list = list(df1['rawContent'])
renderedContent_list = list(df1['renderedContent'])
sentiment_list = list(df1['sentiment'])

rawContent_dict = {tid_list[i]: rawContent_list[i] for i in range(len(tid_list))}
renderedContent_dict = {tid_list[i]: renderedContent_list[i] for i in range(len(tid_list))}
sentiment_dict = {tid_list[i]: sentiment_list[i] for i in range(len(tid_list))}

In [9]:
# use tid to search for tweets in pkl files scraped everyday, to get view, like and retweet count at previous dates

tid_list = list(df1['tid'])

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = datetime.date(2023, 3, 28)
# only to 04-18 since 04-18 is already in the df1 
end_date = datetime.date(2023, 4, 18)
# end_date = datetime.date(2023, 3, 30)

date_ranges = daterange(start_date, end_date)

df_column = ['tid', 'rawContent', 'renderedContent', 'datetime', 'date', 'scrapDate', 'retweetCount', 'viewCount', 'likeCount', 'sentiment']
df2 = df1.copy(deep=True)

for dateobj in date_ranges:
    datestr = dateobj.strftime('%y%m%d')
    file = f'./daily/tweet_verified_221130-{datestr}.pkl'

    with open(file, 'rb') as f:
        tweets_dict1 = pickle.load(f)

    arr = []
    
    for tid, tweet in tweets_dict1.items():
        if tid in tid_list:
            rawContent = rawContent_dict[tid]
            renderedContent = renderedContent_dict[tid]
            _datetime = tweet.date
            _date = _datetime.date()
            scrapDate = dateobj
            retweetCount = tweet.retweetCount
            viewCount = tweet.viewCount if tweet.viewCount else 0
            likeCount = tweet.likeCount if tweet.likeCount else 0
            sentiment = sentiment_dict[tid]
            arr.append((tid, rawContent, renderedContent, _datetime, _date, scrapDate, retweetCount, viewCount, likeCount, sentiment))

    df2 = pd.concat([df2, pd.DataFrame(arr, columns=df_column)])
    
print(len(df2))


1055


In [10]:
df2.head()

,tid,rawContent,renderedContent,datetime,date,scrapDate,retweetCount,viewCount,likeCount,sentiment
0,1641219919202361344,NEW: Prominent Google AI researcher resigned after warning Alphabet CEO Sundar Pichai and other senior execs that Bard—Google’s rival to ChatGPT—was *using data from ChatGPT*. Big no-no in that world. https://t.co/a5NeclJPK5 w/ @jon_victor_ https://t.co/YEZqEqpzPS,NEW: Prominent Google AI researcher resigned after warning Alphabet CEO Sundar Pichai and other senior execs that Bard—Google’s rival to ChatGPT—was *using data from ChatGPT*. Big no-no in that world. theinformation.com/articles/alpha… w/ @jon_victor_ https://t.co/YEZqEqpzPS,2023-03-29 23:24:55+00:00,2023-03-29,2023-04-18,463,1071319,2342,NEGATIVE
1,1641062903510888449,"AI systems like ChatGPT could impact 300 million full-time jobs worldwide, with administrative and legal roles some of the most at risk, Goldman Sachs, $GS, says.","AI systems like ChatGPT could impact 300 million full-time jobs worldwide, with administrative and legal roles some of the most at risk, Goldman Sachs, $GS, says.",2023-03-29 13:01:00+00:00,2023-03-29,2023-04-18,214,299175,979,POSITIVE
2,1641130391825453057,New blog post: The Give-to-Get Model for AI Startups: An Old Trick for Crowdsourcing Proprietary Data Could Work Well for New AI Startups ChatGPT-4 helped me write this post. https://t.co/4Q85AgUmRa,New blog post: The Give-to-Get Model for AI Startups: An Old Trick for Crowdsourcing Proprietary Data Could Work Well for New AI Startups ChatGPT-4 helped me write this post. sacks.substack.com/p/the-give-to-…,2023-03-29 17:29:10+00:00,2023-03-29,2023-04-18,66,264542,584,NEGATIVE
3,1641070970713653248,We want to hear from you! Tell us how you're using ChatGPT in the comments. https://t.co/sU5dSRx7Y2,We want to hear from you! Tell us how you're using ChatGPT in the comments. https://t.co/sU5dSRx7Y2,2023-03-29 13:33:03+00:00,2023-03-29,2023-04-18,27,244625,114,NEGATIVE
4,1641222406504062981,"Fascinating case of how ChatGPT’s capabilities advance I ask it for random numbers, and it uses Wolfram to do it. I for another method &amp; it shows me a Python program. The third try &amp; it makes some up. I ask it to analyze those numbers and it makes a math argument they are random https://t.co/Fyi9Fr8KA7","Fascinating case of how ChatGPT’s capabilities advance I ask it for random numbers, and it uses Wolfram to do it. I for another method &amp; it shows me a Python program. The third try &amp; it makes some up. I ask it to analyze those numbers and it makes a math argument they are random https://t.co/Fyi9Fr8KA7",2023-03-29 23:34:48+00:00,2023-03-29,2023-04-18,66,200803,797,POSITIVE


In [11]:
df2.groupby('date').count()

,tid,rawContent,renderedContent,datetime,scrapDate,retweetCount,viewCount,likeCount,sentiment
date,,,,,,,,,
2023-03-29,100,100,100,100,100,100,100,100,100
2023-03-30,95,95,95,95,95,95,95,95,95
2023-03-31,89,89,89,89,89,89,89,89,89
2023-04-01,86,86,86,86,86,86,86,86,86
2023-04-02,80,80,80,80,80,80,80,80,80
2023-04-03,75,75,75,75,75,75,75,75,75
2023-04-04,70,70,70,70,70,70,70,70,70
2023-04-05,64,64,64,64,64,64,64,64,64
2023-04-06,60,60,60,60,60,60,60,60,60


In [12]:
# reordering dataframe

df2 = df2.sort_values(by=['tid', 'scrapDate'], ascending=True)

In [13]:
# overall first, zoom in later
# plot how the change rate of the value daily

import plotly.express as px

# viewCount_dict = df2[['tid', 'viewCount']]

# df2[['tid', 'viewCount']].groupby('tid')
# viewCount_dict = df2[['tid', 'viewCount']]
temp = list(df2[['tid', 'scrapDate', 'viewCount', 'likeCount', 'retweetCount',]].groupby(['tid']))

fig = go.Figure()
for gp in temp:
    # print(gp[0])

    tid = gp[0]
    viewCount_list = list(gp[1]['viewCount'])
    # likeCount_list = list(gp[1]['likeCount'])
    # retweetCount_list = list(gp[1]['retweetCount'])
    viewCount_rate = np.diff(viewCount_list) # / viewCount_list[:-1] * 100
    # likeCount_rate = np.diff(likeCount_list) / likeCount_list[:-1] * 100
    # retweetCount_rate = np.diff(retweetCount_list) / retweetCount_list[:-1] * 100
    scrapDate_list = list(gp[1]['scrapDate'])[1:]

    fig.add_trace(go.Scatter(x=scrapDate_list, y=viewCount_rate,
                    mode='lines',
                    name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=likeCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=retweetCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    
    # print(viewCount_list)
    # print(likeCount_list)
    # print(retweetCount_list)
    # print(viewCount_rate)
    # print(likeCount_rate)
    # print(retweetCount_rate)
    # print(scrapDate_list)

fig.show()
# print(list(df2[['tid','viewCount']].groupby(['tid']))[0][0])
# print(list(df2[['tid','viewCount']].groupby(['tid']))[0][1]['viewCount'])
# df_vis[['name','text','month']].drop_duplicates()

# df_vis = pd.DataFrame()
# fig = px.line(df_vis, x="date", y="view change rate", title='change rate compared to previous date', color='tid')
# fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_18256\438819588.py:10: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  temp = list(df2[['tid', 'scrapDate', 'viewCount', 'likeCount', 'retweetCount',]].groupby(['tid']))


In [14]:
temp = list(df2[['tid', 'scrapDate', 'viewCount', 'likeCount', 'retweetCount',]].groupby(['tid']))

fig = go.Figure()
for gp in temp:
    # print(gp[0])

    tid = gp[0]
    viewCount_list = list(gp[1]['viewCount'])
    # likeCount_list = list(gp[1]['likeCount'])
    # retweetCount_list = list(gp[1]['retweetCount'])
    viewCount_rate = np.diff(viewCount_list) / viewCount_list[:-1] * 100
    # likeCount_rate = np.diff(likeCount_list) / likeCount_list[:-1] * 100
    # retweetCount_rate = np.diff(retweetCount_list) / retweetCount_list[:-1] * 100
    scrapDate_list = list(gp[1]['scrapDate'])

    fig.add_trace(go.Scatter(x=scrapDate_list, y=viewCount_list,
                    mode='lines',
                    name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=likeCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=retweetCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    
    # print(viewCount_list)
    # print(likeCount_list)
    # print(retweetCount_list)
    # print(viewCount_rate)
    # print(likeCount_rate)
    # print(retweetCount_rate)
    # print(scrapDate_list)

fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_18256\3968809638.py:1: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.

C:\Users\user\AppData\Local\Temp\ipykernel_18256\3968809638.py:11: RuntimeWarning:

divide by zero encountered in divide



In [15]:
# keep cumulative view, like, retweet plot (3 plots) 
# change highest values to obvious, low values line to not obvious, with gradient Done
# hover show tid, value, tweet

In [16]:
# Likes: 00b386
# Views: 6666ff
# Retweets: ff5050

In [23]:
color_tuple = [102, 102, 255]
color_tuple = [255-c for c in color_tuple]

col = 'viewCount'

def to_rgb_gradient(color_tuple, value):
    fac = value/df2[col].max()
    code = '#'+''.join([re.sub('0x', '', hex(255-int(c*fac))).zfill(2) for c in color_tuple])
    return code


fig = go.Figure()
for gp in temp:
    tid = gp[0]
    value_list = list(gp[1][col])
    scrapDate_list = list(gp[1]['scrapDate'])

    fig.add_trace(go.Scatter(x=scrapDate_list,
                             y=value_list,
                            # mode='lines',
                            name='lines',
                            showlegend = False,
                            line=dict(color=to_rgb_gradient(color_tuple, value_list[-1])),
                            hovertemplate =
                            '<b>viewCount</b>: %{y:.0f}'+
                            '<br><b>Date</b>: %{x}'+
                            '<br>%{text}',
                            text=['tid: '+str(tid)+'<br><br>content: '+list(df2[df2['tid'] == tid]['renderedContent'])[0][:150] for i in range(len(value_list))]
                    ))
 
fig.show()

In [18]:
color_tuple = [0, 179, 134]
color_tuple = [255-c for c in color_tuple]

col = 'likeCount'

def to_rgb_gradient(color_tuple, value):
    fac = value/df2[col].max()
    code = '#'+''.join([re.sub('0x', '', hex(255-int(c*fac))).zfill(2) for c in color_tuple])
    return code


fig = go.Figure()
for gp in temp:
    tid = gp[0]
    value_list = list(gp[1][col])
    scrapDate_list = list(gp[1]['scrapDate'])

    fig.add_trace(go.Scatter(x=scrapDate_list,
                             y=value_list,
                            # mode='lines',
                            name='lines',
                            showlegend = False,
                            line=dict(color=to_rgb_gradient(color_tuple, value_list[-1])),
                            hovertemplate =
                            '<b>likeCount</b>: %{y:.0f}'+
                            '<br><b>Date</b>: %{x}'+
                            '<br>%{text}',
                            text=['tid: '+str(tid)+'<br><br>content: '+list(df2[df2['tid'] == tid]['renderedContent'])[0][:150] for i in range(len(value_list))]
                    ))
 
fig.show()

In [19]:
color_tuple = [255, 90, 90]
color_tuple = [255-c for c in color_tuple]

col = 'retweetCount'

def to_rgb_gradient(color_tuple, value):
    fac = value/df2[col].max()
    code = '#'+''.join([re.sub('0x', '', hex(255-int(c*fac))).zfill(2) for c in color_tuple])
    return code


fig = go.Figure()
for gp in temp:
    tid = gp[0]
    value_list = list(gp[1][col])
    scrapDate_list = list(gp[1]['scrapDate'])

    fig.add_trace(go.Scatter(x=scrapDate_list,
                             y=value_list,
                            # mode='lines',
                            name='lines',
                            showlegend = False,
                            line=dict(color=to_rgb_gradient(color_tuple, value_list[-1])),
                            hovertemplate =
                            '<b>retweetCount</b>: %{y:.0f}'+
                            '<br><b>Date</b>: %{x}'+
                            '<br>%{text}',
                            text=['tid: '+str(tid)+'<br><br>content: '+list(df2[df2['tid'] == tid]['renderedContent'])[0][:150] for i in range(len(value_list))]
                    ))
 
fig.show()

In [21]:
temp = list(df2[['tid', 'scrapDate', 'viewCount', 'likeCount', 'retweetCount',]].groupby(['tid']))

fig = go.Figure()
for gp in temp[:]:
    # print(gp[0])

    tid = gp[0]
    # viewCount_list = list(gp[1]['viewCount'])
    # likeCount_list = list(gp[1]['likeCount'])
    retweetCount_list = list(gp[1]['retweetCount'])
    # viewCount_rate = np.diff(viewCount_list) / viewCount_list[:-1] * 100
    # likeCount_rate = np.diff(likeCount_list) / likeCount_list[:-1] * 100
    # retweetCount_rate = np.diff(retweetCount_list) / retweetCount_list[:-1] * 100
    scrapDate_list = list(gp[1]['scrapDate'])

    fig.add_trace(go.Scatter(x=scrapDate_list, y=retweetCount_list,
                    mode='lines',
                    name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=likeCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    # fig.add_trace(go.Scatter(x=scrapDate_list, y=retweetCount_rate,
    #                 mode='lines',
    #                 name='lines'))
    

fig.show()

C:\Users\user\AppData\Local\Temp\ipykernel_18256\4139484508.py:1: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.

